### Part IV: Elastic Net Regression using Gradient Descent [30 points]

In [1]:
import numpy as np
import pandas as pd

In [13]:
diamond_df = pd.read_csv('Data/noisy_datasets/diamond.csv')
diamond_df = diamond_df.drop(columns=['Unnamed: 0'])

In [14]:
print(diamond_df.head())

  carat      cut color clarity  average us salary  \
0  0.23    Ideal     E     SI2              31282   
1  0.21  Premium     E     SI1              40049   
2  0.23     Good     E     VS1              33517   
3  0.29  Premium     I     VS2              38495   
4  0.31     Good     J     SI2              34178   

   number of diamonds mined (millions) depth table price     x     y     z  
0                                 5.01  61.5    55   326  3.95  3.98  2.43  
1                                 1.69  59.8    61   326  3.89  3.84  2.31  
2                                 3.85  56.9    65   327  4.05  4.07  2.31  
3                                 3.49  62.4    58   334   4.2  4.23  2.63  
4                                 4.70  63.3    58   335  4.34  4.35  2.75  


In [15]:
diamond_df.describe()

,average us salary,number of diamonds mined (millions)
count,53940.000000,53940.000000
mean,39521.990100,2.902669
std,5486.892971,1.325985
min,30000.000000,0.600000
25%,34780.000000,1.750000
50%,39547.500000,2.910000
75%,44252.000000,4.050000
max,48999.000000,5.200000


In [17]:
print("Shape of the dataset is: ", diamond_df.shape)

Shape of the dataset is:  (53940, 12)


In [18]:
diamond_df.isna().sum() #checking for empty columns

carat                                  1510
cut                                    1293
color                                  1512
clarity                                 353
average us salary                         0
number of diamonds mined (millions)       0
depth                                   694
table                                  1542
price                                  1583
x                                      1526
y                                      1221
z                                      1433
dtype: int64